In [ ]:
# default_exp scraper

# The Guardian Scraper

> Scraping Premier League Previews from the Guardian.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re
from time import sleep
import numpy as np
import pandas as pd

In [ ]:
#export
class Parser:
    
    @staticmethod
    def parse_page(url, session): # -> (html document)
        # Request the url
        request = session.get(url)
        # Get the html document of the page
        page = BeautifulSoup(request.text, 'html.parser')
        return page

    @staticmethod
    def get_next_page(page): # -> (string,Boolean)
        # If we are at the last page , last_page = True else last_page = False
        last_page = False
        # Pick up the pagination HTML part
        pagination_section = page.find('div', {'class': 'pagination__list'})
        # If we don't find the "next" button (it's the last page)
        # We are in the last page
        if not page.find('a', {'rel': 'next'}):
            #We pick up the number of the page and we return the link
            page_number = page.find('span', {'aria-label': 'Current page'}).text
            url = "https://www.theguardian.com/football/series/match-previews?page="+page_number
            last_page = True
            return url , last_page
        # If it's not the last page, we pick up the link of the following page
        else:
            url = page.find('a', {'rel': 'next'})['href']
            return url, last_page


In [ ]:
#export
class PageExtractor:
    
    @staticmethod
    def get_values_matching_regex(page,regex): # -> list(string)      
        
        # All Information are located in the "p tag" of html
        # We pick up all the p tags
        for paragraph in page.find_all('p',{'class':'dcr-bixwrd'},limit=9): 
            # We pick up the string values located in the paragraph
            # For "odds" information, if we find Evens, we replace it by 1-1
            section = paragraph.text.replace("Evens","1-1").replace("Evs","1-1").replace("evens","1-1")
            # To extract our information regex pattern 
            # To ignore case sensitivity we use re.I  
            pattern = re.compile(regex,re.I)
            # If there is a match we return the list of values , else we return an empty array
            if(pattern.findall(section)):
                return pattern.findall(section)

    @staticmethod
    def extract_teams_names(title): # -> dict({string:string,string:string})

            # 2 possible formats for previews title
            # For example {Squad sheets: Team A v Team B} or {{Team A v Team B : match preview}}
            # We remove text before or after team names
            preview_title = title.replace(': match preview','').replace('Squad sheets:','')
            # Names are located in the title of the preview
            # Home team
            home_team = preview_title.split(' v ')[0].strip()
            # Away team
            away_team = preview_title.split(' v ')[1].strip()
            # we return names
            names = dict({"home": home_team, "away": away_team})
            return names

    @staticmethod
    def extract_text_authors(page): # -> dict({string:string,string:string})
      
            # Preview may not have text and author,
            # We initialize author and text to 'n/a' (not available), 
            author='n/a'
            text='n/a'
            # Text preview is located in the first p tag of the page
            text_section = page.find('p', {'class': 'dcr-bixwrd'})
            text_value = text_section.text
            # If the length of the text is under 50
            # Probably there's no text in the preview
            if len(text_value) > 50:
                text = text_value
                # Author name is locate in the "Strong" tag under the "p" tag
                author_section = page.select_one("p > strong")
                if(str(author_section) != 'None'):
                    author = author_section.text 

            preview_text_author = dict({"text": text, "author": author})
            return preview_text_author

    @staticmethod
    def extract_match_infos(page, venue_regex, referee_regex, odds_regex): # -> dict({string:string,string:list(string,string,string)})
  
            # Extract venue, referee and odds values
            venue = PageExtractor.get_values_matching_regex(page,venue_regex)[0].strip()
            referee = PageExtractor.get_values_matching_regex(page,referee_regex)[0].strip()
            odds = PageExtractor.get_values_matching_regex(page,odds_regex)
            match_infos = dict({"venue": venue, "referee": referee, "odds": odds})
            return match_infos


In [ ]:
#export
class ScrapingTheGuardian:
    
    # venue, referee, odds pattern regex
    venue_regex = 'Venue(.*)' 
    referee_regex = 'Referee(.*)'
    odds_regex = 'Odds[\s]*.*[\s]+(\d{1,3}-\d{1,3})[\s]*.*[\s]+(\d{1,3}-\d{1,3})[\s]*.*[\s]+(\d{1,3}-\d{1,3})'
  
    def __init__(self):
        
        # Initialize session to start scraping
        self.session = HTMLSession()
  
    @staticmethod
    def calculate_betting_odds(odds): # -> dict({string:float,string:float,string:float})
    
        # Initialize betting odds to n/a (not available)
        # Some previews may not include odds
        odds_home = "n/a"
        odds_away = "n/a"
        odds_draw = "n/a"

        if(len(odds) > 0): # If odds exist 
            # example of odds {H 4-6 A 43-10 D 3-1} {liverpool 4-6 Tottenham 43-10 Draw 3-1} {H 4-6 43-10 D 3-1}
            # The formula will be (4/6)+1 , (43/10)+1 , (3/1)+1
            if len(odds[0]) == 3:
                # Home team odds
                betting_odds_home = odds[0][0]
                odds_home = (int(betting_odds_home.split('-')[0]) / int(betting_odds_home.split('-')[1]) ) + 1
                # Away team odds
                betting_odds_away = odds[0][1]
                odds_away = (int(betting_odds_away.split('-')[0]) / int(betting_odds_away.split('-')[1])) + 1
                # Draw odds
                betting_odds_draw = odds[0][2]
                odds_draw = (int(betting_odds_draw.split('-')[0]) / int(betting_odds_draw.split('-')[1])) + 1

        betting_odds = dict({"odds_home": odds_home, "odds_away": odds_away, "odds_draw": odds_draw})
        return betting_odds

    @staticmethod
    def extract_preview_items(page,title):#=> dict({items}) 
        
        # meth1: extract team names
        names = PageExtractor.extract_teams_names(title)
        # Home team and  Away Team
        home_team  = names["home"]
        away_team  = names["away"]
        # meth2: extract text and author of the preview
        text_author = PageExtractor.extract_text_authors(page)
        text = text_author["text"]
        author = text_author["author"]
        # meth3: extract match infos (venue,referee,odds)
        match_infos = PageExtractor.extract_match_infos(page,ScrapingTheGuardian.venue_regex,ScrapingTheGuardian.referee_regex,ScrapingTheGuardian.odds_regex)
        venue = match_infos["venue"]
        referee = match_infos["referee"]
        odds = match_infos["odds"]
        # meth4: calculate betting odds
        betting_odds = ScrapingTheGuardian.calculate_betting_odds(odds)
        # Home team betting odds
        odds_home_team = betting_odds["odds_home"]
        # Away team betting odds
        odds_away_team = betting_odds["odds_away"]
        # Draw betting odds
        odds_draw = betting_odds["odds_draw"]
        # Return preview items
        preview_items = dict({"home_team": home_team, "away_team": away_team, "text": text, "author": author, "venue": venue, "referee": referee, "odds":odds[0], "odds_home_team": odds_home_team, "odds_away_team": odds_away_team, "odds_draw": odds_draw}) 
        return preview_items

    def extract_previews(self,soup):# -> list(dict(items))
        
        all_previews_information=[]
        # We pick up all of the match previews on the webpage.
        previews = soup.findAll('div',{'class':'fc-item__content'})
        # for each preview we extract its information
        for preview in previews:
            preview_items={}
            # Pick up the preview link
            preview_link = preview.find("a")["href"]
            # Pick up the match preview page
            preview_page = Parser.parse_page(preview_link,self.session)
            # We need only Premier League Previews
            # To filter previews we need to Find the title of the preview 
            preview_title = preview_page.find('h1').text
            # Check if "cup" exists in the title or in the link
            # If the preview is not for a cup competition , we proceed the extraction
            cup_not_in_title = re.search('cup', preview_title , re.IGNORECASE) is None
            cup_not_in_link  = re.search('cup',preview_link, re.IGNORECASE) is None
            if (cup_not_in_title and cup_not_in_link):
                preview_items = ScrapingTheGuardian.extract_preview_items(preview_page,preview_title)
                all_previews_information.append(preview_items)
    
        return all_previews_information


In [ ]:
url = "https://www.theguardian.com/football/series/match-previews"
scraper = ScrapingTheGuardian()
previews_infos=[]
page = Parser.parse_page(url,scraper.session)
previews_infos = scraper.extract_previews(page)

In [ ]:
pd.DataFrame(previews_infos)

,home_team,away_team,text,author,venue,referee,odds,odds_home_team,odds_away_team,odds_draw
0,Leeds,Newcastle,Can Eddie Howe choreograph Newcastle’s second ...,Louise Taylor,Elland Road,Chris Kavanagh,"(1-1, 3-1, 5-2)",2.000000,4.000000,3.500000
1,West Ham,Leeds,West Ham regained fourth place by beating Norw...,Jacob Steinberg,London Stadium,Mike Dean,"(4-6, 43-10, 3-1)",1.666667,5.300000,4.000000
2,Tottenham,Arsenal,For the first time in a while more than just b...,Stephen Hollis,Tottenham Hotspur Stadium,Chris Kavanagh,"(8-5, 2-1, 12-5)",2.600000,3.000000,3.400000
3,Liverpool,Brentford,"Albert Stubbins, the only footballer to featur...",Andy Hunter,Anfield,Jon Moss,"(1-3, 21-2, 5-1)",1.333333,11.500000,6.000000
4,Newcastle,Watford,Moussa Sissoko returns to Newcastle in Watford...,Louise Taylor,St James’ Park,Paul Tierney,"(19-17, 27-10, 27-10)",2.117647,3.700000,3.700000
5,Manchester City,Chelsea,Manchester City would surely finish off Chelse...,Jamie Jackson,Etihad Stadium,Craig Pawson,"(8-11, 22-5, 3-1)",1.727273,5.400000,4.000000
6,Aston Villa,Manchester United,The extent of the problems Ralf Rangnick has i...,Stephen Hollis,Villa Park,David Coote,"(2-1, 19-13, 13-5)",3.000000,2.461538,3.600000
7,Norwich,Everton,If Norwich have any hope of staging an unlikel...,Stephen Hollis,Carrow Road,Andy Madley,"(37-13, 19-17, 18-7)",3.846154,2.117647,3.571429
8,Wolves,Southampton,Southampton were excellent in their 4-1 stroll...,Graham Searles,Molineux,Michael Salisbury,"(11-8, 5-2, 9-4)",2.375000,3.500000,3.250000
9,Brentford,Aston Villa,Steven Gerrard returns to the touchline for As...,Tim Knowles,Brentford Community Stadium,Craig Pawson,"(2-1, 6-4, 2-1)",3.000000,2.500000,3.000000
